### Michelson contrast computation

In [2]:
from __future__ import annotations
from typing import Iterable, Tuple, Union

Color = Union[str, Iterable[float]]

def _parse_rgb(color: Color) -> Tuple[float, float, float]:
    if isinstance(color, str):
        s = color.strip()
        if s.startswith("#"):
            s = s[1:]
        if len(s) == 3:  
            s = "".join(ch*2 for ch in s)
        if len(s) != 6:
            raise ValueError("Hex colors must be like '#RRGGBB' or '#RGB'.")
        r = int(s[0:2], 16) / 255.0
        g = int(s[2:4], 16) / 255.0
        b = int(s[4:6], 16) / 255.0
        return (r, g, b)
    else:
        r, g, b = list(color)
        scale = 255.0 if max(r, g, b) > 1.0 else 1.0
        return (r / scale, g / scale, b / scale)

def _srgb_to_linear(c: float) -> float:
    return c / 12.92 if c <= 0.04045 else ((c + 0.055) / 1.055) ** 2.4

def relative_luminance(color: Color) -> float:
    """
    Relative luminance Y (0=black, 1=reference white) per sRGB/Rec.709.
    """
    r, g, b = _parse_rgb(color)
    R = _srgb_to_linear(r)
    G = _srgb_to_linear(g)
    B = _srgb_to_linear(b)
    # Rec. 709 luma coefficients
    return 0.2126 * R + 0.7152 * G + 0.0722 * B

def michelson_contrast(color1: Color, color2: Color) -> float:
    """
    Michelson contrast for a binary grating made of two colors:
        C = (I_max - I_min) / (I_max + I_min)
    where I is the relative luminance of each color.
    """
    I1 = relative_luminance(color1)
    I2 = relative_luminance(color2)
    Imax, Imin = (I1, I2) if I1 >= I2 else (I2, I1)
    denom = Imax + Imin
    if denom == 0.0:
        return 0.0  
    return (Imax - Imin) / denom


In [3]:
# Yellow + Red + Blue
michelson_contrast("#b1754b", "#323233")

0.7518367688886185

In [4]:
# Blue + Red
michelson_contrast("#8a3867", "#323233")

0.4845236817665021

In [5]:
# Red + Black
michelson_contrast("#8a1819", "#323233")

0.31403535822668177

In [6]:
# Red
michelson_contrast("#fe1819", "#323233")

0.7440579760672751

In [7]:
# Blue
michelson_contrast("#1859b2", "#323233")

0.5348434570604965

In [8]:
# White
michelson_contrast("#ffffff", "#323233")

0.938015850607256